<a href="https://colab.research.google.com/github/msrepo/ml-mscise-2023/blob/master/Lecture8_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 8: Naive Bayes

Adapted from/ Original Notebook by:
**Volodymyr Kuleshov**
*Applied Machine Learning*



**Part 1: Text Classification**

We will now do a quick detour to talk about an important application area of machine learning text classification. 

Afterwards, we will see how text classification motivates new classification algorithms.

An interesting instance of a classification problem is classifying text.
-  includes a lot of applied problems: spam filtering, fraud detection, medical record classification etc.
- input $x$ are sequences of words of arbitrary length
- The dimensionality of text inputs is usually very large, proportional to the size of the vocabulary

**Classification Dataset: Twenty Newsgroups**

To illustrate the text classification problem, we will use a popular dataset called *20-newsgroups*

- It contains ~20,000 documents collected approximately evenly from 20 different online newsgroups
- Each newgroup covers a different topic such as medicine, computer graphics, or religion
- This dataset is widely used to benchmark text classification and other types of algorithms

Lets load this dataset.


In [24]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

# for this lecture, we will restrict our attention to just 4 different newsgroups
categories = ['misc.forsale', 'rec.motorcycles', 'comp.graphics', 'sci.med']

# load the dataset
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=0)

print(twenty_train.DESCR[:1000])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [25]:
print(twenty_train.target_names)
print(twenty_train.data[0])

['comp.graphics', 'misc.forsale', 'rec.motorcycles', 'sci.med']
Subject: Ovarian cancer treatment centers
From: <RBPRMA@rohvm1.rohmhaas.com>
Organization: Rohm and Haas Company
Lines: 9

A relative of mine has recently been diagnosed with "stage 3 papillary cell
ovarian cancer".  We are urgently seeking the best place in the country for
treatment for this.

Does anyone have any suggestions?

As you might suspect, time is of the essence.

Thanks for your help.                                      Bob



**Feature Representations for Text**

Each data point $x$ in this dataset is a sequence of characters of an arbitrary length.

How do we transform these into a $d$-dimensional features $\phi(x)$ that can be used with our machine learning algorithms?

- we may devise hand-crafted features by inspecting the data:
  - Does the message contain the word "bike"? Does the email of the user belong to a university?
- We can count the number of occurrences of each word:
  - Does this message contain "Apple", yes or no?
- Finally, many modern deep learning methods can directly work with sequences of characters of an arbitrary length.


**Bag of Words Representations**

Perhaps the most widely used approach to representing text documents is called "bag of words".

We start by defining a vocabulary $V$ containing all the possible words we are interested in e.g. $$V = \{ \text{cancer}, \text{slow}, \ldots\}$$

A bag of words representation of a document $x$ is a function $\phi(x) \to \{0,1\}^{|V|}$ that outputs a feature vector $\phi(x) = \left(\begin{array}{c} 0 \\ 1 \\ 0 \\ \vdots \\ 0 \\\end{array}\right)$ of dimension $|V|$. The $j^{th}$ component $\phi(x)_j$ equals $1$ if $x$ contains the $j$th word in $V$ and $0$ otherwise.

Lets see an example of this approach on "20-newsgroups".

We start by computing these features using the *sklearn* library.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize the training set
count_vect = CountVectorizer(binary=True)
X_train = count_vect.fit_transform(twenty_train.data)
X_train.shape

(2361, 35807)

In *sklearn*, we can retrieve the index of $\phi(x)$ associated with each *word* using the expression `count_vect.vocabulary_.get(word)`

In [27]:
print('Index for the word "cancer"',count_vect.vocabulary_.get(u'cancer'))
print('Index for the word "computer"',count_vect.vocabulary_.get(u'computer'))

Index for the word "cancer" 8612
Index for the word "computer" 10160


Our featurized dataset is in the matrix `X_train`. We can use the above indices to retrieve the 0-1 value that has been computed for each word.

In [28]:
# we can examine if any of these words are present in our previous data point
print(twenty_train.data[0])

# lets see if it contans these two words?
print('----'*20)
print('Value at the index for the word "treatment"',X_train[0,count_vect.vocabulary_.get(u'treatment')])
print('Value at the index for the word "cancer"',X_train[0, count_vect.vocabulary_.get(u'cancer')])

Subject: Ovarian cancer treatment centers
From: <RBPRMA@rohvm1.rohmhaas.com>
Organization: Rohm and Haas Company
Lines: 9

A relative of mine has recently been diagnosed with "stage 3 papillary cell
ovarian cancer".  We are urgently seeking the best place in the country for
treatment for this.

Does anyone have any suggestions?

As you might suspect, time is of the essence.

Thanks for your help.                                      Bob

--------------------------------------------------------------------------------
Value at the index for the word "treatment" 1
Value at the index for the word "cancer" 1


**Practical Considerations**

In practice, we may use some additional modifications of this technique.

- Sometimes, the feature $\phi(x)_j$ for the j-th word holds the count of word $j$ instead of just the binary occurence
- The raw text is usually preprocessed. One common technique is *stemming*, in which we only keep the rrot of the word.
 - e.g. 'slowly' 'slowness', both map to 'slow
- Filtering for common *stopwords* such as 'the', 'a', 'and'. Similarly, rare words are also typically excluded.



**Classification using BoW features**

Lets now have a look at the performance of classification over bag of words features.

Now that we have a feature representation $\phi(x)$, we can apply the classifier of our choice, such as logistic regression.

In [16]:
from sklearn.linear_model import LogisticRegression

# Create an instance of Softmax and fit the data
logreg = LogisticRegression(C=1e5,multi_class='multinomial',verbose=True)
logreg.fit(X_train, twenty_train.target)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s finished


LogisticRegression(C=100000.0, multi_class='multinomial', verbose=True)

In [23]:
docs_new = ['Long COVID has a lot of long-term effects.', 
            'New baby shoes. never worn.',
           ]

X_new = count_vect.transform(docs_new)
predicted = logreg.predict(X_new)

for doc, category in zip(docs_new, predicted):
  print(f'{doc} => {twenty_train.target_names[category]}')

Long COVID has a lot of long-term effects. => sci.med
New baby shoes. never worn. => misc.forsale


**Summary of Text Classification**

- Classifying text normally requires specifying features over the raw data.

- A widely used representation is 'bag of words', in which features are occurences or counts of words.

-  Once text if featured, any off-the-shelf supervised learning algorithm can be applied, but some work better than others, as we will see next.

## Part 2: Naive Bayes
Next, we are going to look at Naive Bayes - a generative classification algorithm. We will apply Naive Bayes to the text classification problem.

**Review: Generative Models**

There are two types of probabilistic models: *generative* and *discriminative*.

$$
\text{Generative Model};  P_\theta(x,y):\mathcal{X} \times \mathcal{Y} \to [0,1] \\
\text{discriminative Model}; P_\theta(y|x):\mathcal{X} \times \mathcal{Y} \to [0,1]
$$

Given a new datapoint $x'$, we can match it against each class model and find the class that looks most similar to it:

$$
 \arg\max_y \log p(y|x) = \arg \max_y \log \frac{p(x|y)p(y)}{p(x)} = \arg \max_y \log p(x|y) p(y)
$$

where we have applied Bayes rule in the second equation.


**Review: Gaussian Discriminant Model**

The GDA algorithm defines the following model family:

- the probability $P(x|y=k)$ of the data under class $k$ is a multivariate Gaussian $\mathcal{N}(x;\mu_k,\textstyle \sum_k)$ with parameters $\mu_k$, $\sum_k$.

- the distribution over classes is Categorical, denoted $\text{Categorical}(\phi_1,\phi_2,\ldots,\phi_n)$. Thus, $P_y(y=k) = \phi_k$. Thus, $P_{\theta}(x,y)$ is a mixture of $K$ Gaussians.

$$
 P_{\theta}(x,y) = \sum_{k=1}^{K}P_{\theta}(y=k)P_{\theta}(x|y=k) = \sum_{k=1}^{K}\phi_k\mathcal{N}(x;\mu_k,\sum_k)
$$

**Naive Bayes Assumption**
In order to deal with high-dimensional $x$, we simplify the problem by making the *Naive Bayes* assumption.

$$
p(x|y) = \prod_{j=1}^dp(x_j|y)
$$

In other words, the probability $p(x|y)$ factorizes over each dimension.

- For example, if $x$ is a binary bag of words representation, then $p(x_j|y)$ is the probability of seeing the j-the word.

- We can model each $p(x_j|y)$ via a bernoulli distribution, which has only one parameter.

- Hence, it takes only $d$ parameters (instead of $2^d-1$) to specify the entire distribution $p(x|y) = \prod_{j=1}^dp(x_j|y)$

**Bernoulli Naive Bayes Model**

We can apply the Naive Bayes assumption to obtain a model for when $x$ is in a bag of words representation.

**Review: Maximum Likelihood Learning**

In order to fit probabilistic models, we use the following objective:

$$
 \max_\theta \mathbb{E}_{x,y \sim \mathbb{P}_{data}}\log P_\theta(x,y)
$$

This seeks to find a model that assigns high probability to the training data.

Let's use maximum likelihood to fit the Bernoulli Naive Bayes model. Note that model parameters $\theta$ are the union of the parameters of each sub-model.

$$
\theta = ( \phi_1,\phi_2,\ldots,\phi_K, \psi_{11},\psi_{21},\ldots, \psi_{dK} )
$$

**Learning a Bernoulli Naive Bayes Model**

Given a dataset $\mathcal{D} = \{(x^{(i)},y^{(i)})|i=1,2,\ldots,n \}$, we want to optimize the log-likelihood $l(\theta) = \log L(\theta)$

In [29]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

# for this lecture, we will restrict our attention to just 4 different newsgroups
categories = ['misc.forsale', 'rec.motorcycles', 'comp.graphics', 'sci.med']

# load the dataset
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=0)



In [32]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize the training set
count_vect = CountVectorizer(binary=True, max_features=1000)
y_train = twenty_train.target
X_train = count_vect.fit_transform(twenty_train.data).toarray()
X_train.shape

(2361, 1000)

Lets compute the maximum likelihood model parameters on our dataset.

In [39]:
n = X_train.shape[0]
d = X_train.shape[1]
k = len(categories)
print(f'n {n} d {d} k {k}')

# these are the shapes of the parameters
psis = np.zeros([k,d])
phis = np.zeros([k])

# we can now compute the parameters
for idx in range(k):
  X_k = X_train[y_train==idx]
  psis[idx] = np.mean(X_k,axis=0)
  phis[idx] = X_k.shape[0] / float(n)

# print out class proportions
print(r'ϕ',phis)
print(r'ψ',psis)



n 2361 d 1000 k 4
ϕ [0.24735282 0.24777637 0.25328251 0.25158831]
ψ [[0.03424658 0.01883562 0.0890411  ... 0.4640411  0.16267123 0.0119863 ]
 [0.14871795 0.04957265 0.19316239 ... 0.46666667 0.19487179 0.00854701]
 [0.03344482 0.02341137 0.09698997 ... 0.63545151 0.31772575 0.04013378]
 [0.03198653 0.03198653 0.11111111 ... 0.54882155 0.29292929 0.02020202]]


We can now compute predictions using Bayes rule.

In [43]:
def naive_bayes_predict(x, psis, phis):
  """This returns class assignments and scores under the NB model.

  We compute \arg \max_y p(y|x) as \arg \max_y p(x|y)p(y)
  """
  # adjust shapes
  n, d = x.shape
  x =np.reshape(x,(1,n,d))
  psis = np.reshape(psis, (k,1,d))

  # clip probabilities to avoid log(0)
  psis = psis.clip(1e-14, 1-1e-14)

  # compute log probabilities
  logpy = np.log(phis).reshape([k,1])
  logpxy = x * np.log(psis) + (1-x)*np.log(1-psis)
  logpyx = logpxy.sum(axis=2) + logpy

  return logpyx.argmax(axis=0).flatten(), logpyx.reshape([k,n])

idx, logpyx = naive_bayes_predict(X_train, psis, phis)
print('Predictions',idx[:10])
acc = (idx == y_train).mean()
print(f'Accuracy: {acc:.2f}')

Predictions [0 1 3 1 0 2 2 0 1 0]
Accuracy: 0.86
